In [1]:
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import pandas as pd
import numpy as np

In [24]:
from sklearn.linear_model import LinearRegression

df = pd.read_csv(r"C:\Users\Jurgen\Desktop\FYP\Shared\News_Twitter_Stock_NaN.csv")
df = df.dropna().reset_index(drop=True)
# df = df.fillna(0)
X = df[["News_Senti", "Twitter_Senti"]].to_numpy()#.reshape(-1, 1)
y = df["Stock_Difference"].to_numpy()
df

,Date,News_Senti,Close_USD,Stock_Difference,Twitter_Senti
0,9/8/2018,0.101774,41.887453,0.121431,-0.1027
1,5/9/2018,0.179139,41.525001,-1.112296,0.0000
2,24/10/2018,0.219228,37.532583,-0.308136,0.0000
3,26/10/2018,0.072399,35.831099,-0.273189,0.1222
4,8/11/2018,0.188299,39.441578,0.146884,1.6313
...,...,...,...,...,...
76,22/10/2019,0.194338,43.639371,0.707001,0.0000
77,23/10/2019,0.202612,43.691557,0.052186,0.0000
78,24/10/2019,0.112101,43.221990,-0.469567,0.8140
79,28/10/2019,0.107508,43.848824,0.522953,0.6369


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
# GenSenti = []
# for i in range(len(df)):
#     if df.Stock_Difference[i] > 0:
#         GenSenti.append("Increase")
#     elif df.Stock_Difference[i] < 0:
#         GenSenti.append("Decrease")
#     else:
#         GenSenti.append("Constant")
# df["Stock_Changess"] = GenSenti
# df.to_csv(r"C:\Users\Jurgen\Desktop\FYP\Shared\News_Twitter_Stock_Data.csv")


In [26]:
from sklearn.metrics import mean_squared_error, r2_score


regr = LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))
# The intercept
print("Intercept: ", regr.intercept_)

Coefficients: 
 [0.25059388 0.1288344 ]
Mean squared error: 0.29
Coefficient of determination: 0.03
Intercept:  -0.10301323030921938


In [27]:
from sklearn.ensemble import RandomForestRegressor

text_classifier = RandomForestRegressor(n_estimators=500, random_state=0)
text_classifier.fit(X_train, y_train)
y_pred = text_classifier.predict(X_test)


print("R^2: ", metrics.r2_score(y_test, y_pred))
print("Variance Score: ", metrics.explained_variance_score(y_test, y_pred))
print("Max Error: ", metrics.max_error(y_test, y_pred))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R^2:  -0.06596018305959594
Variance Score:  -0.0582354820945834
Max Error:  1.4956616269200005
Mean Absolute Error: 0.4158996058431588
Mean Squared Error: 0.31433367719225186
Root Mean Squared Error: 0.560654686230528


In [28]:
from sklearn import metrics
from sklearn import neighbors


knn = neighbors.KNeighborsRegressor(n_neighbors = 2)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)


print("R^2: ", metrics.r2_score(y_test, y_pred))
print("Variance Score: ", metrics.explained_variance_score(y_test, y_pred))
print("Max Error: ", metrics.max_error(y_test, y_pred))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R^2:  -0.8998458476498763
Variance Score:  -0.8430163682719896
Max Error:  1.3606547549999999
Mean Absolute Error: 0.6445348345238096
Mean Squared Error: 0.5602324935591227
Root Mean Squared Error: 0.7484868025283564


In [ ]:
import matplotlib.pyplot as plt

# Plot outputs
# plt.scatter(TwitterSenti, y_test,  color='yellow')
# plt.scatter(TwitterSenti, y_pred, color='blue')
# plt.scatter(NewsSenti, y_pred, color='purple')
# plt.plot(X_test, y_test, color = "blue")
# plt.scatter(X_test, y_pred, color = "yellow")


plt.xticks(())
plt.yticks(())

plt.show()

# print(X_test.shape, y_test.shape)
# print(X_test.shape, y_pred.shape)

# print(y_pred)



In [ ]:
test = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
test

In [ ]:
NewsSenti = []
TwitterSenti = []
for i in range(len(y_test)):
    NewsSenti.append(X_test[i][0])
    TwitterSenti.append(X_test[i][1])
    

# CLASSIFIER

In [ ]:
#Classifier

# df1 = pd.read_csv(r"C:\Users\Jurgen\Desktop\FYP\News\SK-Learn\News_Stock_Data_30.csv")
# X = df1["News_Senti"].to_numpy().reshape(-1, 1)
# y = df1["Stock_Changes"].to_numpy()
# df1

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
count = 0
for i in range(len(y_test)):
    if y_test[i] == y_pred[i]:
        count += 1
print(count, '/', len(y_test))
print(y_test)
print(y_pred)